https://www.kaggle.com/brendan45774/chess-piece-dectection

In [1]:
pip install imageai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Conda\python.exe -m pip install --upgrade pip


In [2]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Conda\python.exe -m pip install --upgrade pip


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Conda\python.exe -m pip install --upgrade pip


In [4]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Conda\python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import os
import shutil
from pathlib import Path

import imageai
import cv2
from imageai.Detection import ObjectDetection
from imageai.Detection.Custom import DetectionModelTrainer
from imageai.Detection.Custom import CustomObjectDetection

ImportError: cannot import name 'ImageReadMode' from 'torchvision.io' (C:\Conda\lib\site-packages\torchvision\io\__init__.py)

In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("../input/gb-pretrainedyolov3h5/pretrained-yolov3.h5")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image='../input/chess-piece-dectection/images/chess0.png',
                                             output_image_path='./detected.png',
                                             minimum_percentage_probability=30)

for eachObject in detections:
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")

In [ ]:
from PIL import Image
Image.open('./detected.png')

In [ ]:
root_annots_path = '../input/chess-piece-dectection/annotations/'
root_images_path = '../input/chess-piece-dectection/images/'

annots_path = sorted([i for i in Path(root_annots_path).glob('*.xml')])
images_path = sorted([i for i in Path(root_images_path).glob('*.png')])

n_imgs = len(images_path)

In [ ]:
with open(annots_path[6], 'r') as f:
    print(f.read())

In [ ]:
set_classes = set()

for annot in annots_path:
    with open(annot, 'r') as f:
        string = f.read()
        name = string[string.find('<name>')+6:string.find('</name>')]
        set_classes.add(name)

classes = np.array(list(set_classes))
print(classes)

In [ ]:
os.makedirs('../imageai/data/train/images', exist_ok=True)
os.makedirs('../imageai/data/train/annotations', exist_ok=True)

os.makedirs('../imageai/data/validation/images', exist_ok=True)
os.makedirs('../imageai/data/validation/annotations', exist_ok=True)

os.makedirs('../imageai/data/test/images', exist_ok=True)
os.makedirs('../imageai/data/test/annotations', exist_ok=True)

In [ ]:
n_imgs = 81
n_split = n_imgs // 6


for i, (annot_path, img_path) in enumerate(zip(annots_path, images_path)):    
    if i > n_imgs:
        break
    # train-val-test split
    if i < n_split:
        shutil.copy(img_path, '../imageai/data/test/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '../imageai/data/test/annotations/' + annot_path.parts[-1])
    elif n_split <= i < n_split*2:
        shutil.copy(img_path, '../imageai/data/validation/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '../imageai/data/validation/annotations/' + annot_path.parts[-1])
    else:
        shutil.copy(img_path, '../imageai/data/train/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '../imageai/data/train/annotations/' + annot_path.parts[-1])

In [ ]:
print(len(list(Path('../imageai/data/train/annotations/').glob('*.xml'))))
print(len(list(Path('../imageai/data/validation/annotations/').glob('*.xml'))))
print(len(list(Path('../imageai/data/test/annotations/').glob('*.xml'))))

In [ ]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="../imageai/data/")
trainer.setTrainConfig(object_names_array=classes,
                       batch_size=8,
                       num_experiments=30,
                       train_from_pretrained_model="../input/gb-pretrainedyolov3h5/pretrained-yolov3.h5")

trainer.trainModel()

In [ ]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="../imageai/data/")

last_model_name = sorted(os.listdir("../imageai/data/models/"))[-1]
metrics = trainer.evaluateModel(model_path=f"../imageai/data/models/{last_model_name}",
                                json_path="../imageai/data/json/detection_config.json",
                                iou_threshold=0.2,
                                object_threshold=0.3,
                                nms_threshold=0.5)

In [ ]:
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(f"../imageai/data/models/{last_model_name}")
detector.setJsonPath("../imageai/data/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(minimum_percentage_probability=50,
                                             input_image="../imageai/data/validation/images/chess20.png",
                                             output_image_path="../imageai/detected.jpg")
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

In [ ]:
from PIL import Image
Image.open('../imageai/detected.jpg')